In [10]:
from Crawler.LogIn import *

import pandas as pd
import numpy as np


from datetime import datetime
from datetime import date
from datetime import timedelta

from selenium import webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.keys import Keys

In [12]:
# Define user agent
user_agent = "user-agent=Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36"

# Define chromedriver path
executable_path = '/chromedriver'

browser, chrome_options = createBrowser(executable_path, user_agent = user_agent)

url = 'https://www.hkexnews.hk/sdw/search/searchsdw_c.aspx'

# Browse url
browser = getURL(browser, url)

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [C:\Users\Timothy.lo\.wdm\drivers\chromedriver\win32\85.0.4183.87\chromedriver.exe] found in cache


>> 'incognito' not found in keyword arguments
>> Default: Private mode is set in Chrome agent
>> 'disable_extensions' not found in keyword arguments
>> Default: Chrome agent is disabled its extensions
>> user_agent is added to chrome_options 
>> incognito is added to chrome_options 
>> disable_extensions is added to chrome_options 
 
>> browser response: Get https://www.hkexnews.hk/sdw/search/searchsdw_c.aspx


In [37]:
def calendar_click(to_search, class_name):
    
    try:
        calendar_element = browser.find_element_by_class_name(class_name)
        
        for calendar in calendar_element.find_elements_by_tag_name('li'):
            if calendar.text.strip() == to_search:
                
                calendar.click()
        
    except Exception as e:
        print(e)

In [36]:
# Retrieve a list of HK stock
# Define the year, month and day to search
# date_to_search = date.today() - timedelta(1)
date_to_search = date(2019, 12, 11)
year_to_search = str(date_to_search.year)
month_to_search = str(date_to_search.month)
day_to_search = str(date_to_search.day)

def enter_search_kw(date_to_search, stock_code):

    # Select the correct year, month and day
    calendar_click(year_to_search, 'year')
    calendar_click(month_to_search, 'month')
    calendar_click(day_to_search, 'day')

    time.sleep(1)

    # Set Stock Code
    browser.find_element_by_name('txtStockCode').send_keys(stock_code)

    # Click Search
    browser.find_element_by_id('btnSearch').click()

    # Break time
    time.sleep(1)


In [64]:
# Scrape 市場中介者/願意披露的投資者戶口持有人的紀錄 on the page
def Scraper(browser, stock_code, date_to_search):

    # Get search result table
    search_table = browser.find_element_by_class_name('search-details-table-container').find_element_by_tag_name('table')
    
    # Get column_name
    column_name = [col.text for col in search_table.find_elements_by_tag_name('th')]
    
    # Get Search body
    search_body = search_table.find_element_by_tag_name('tbody')
    
    # Get Search result data
    data = np.array([[cell.text for cell in row.find_elements_by_class_name('mobile-list-body')] for row in search_body.find_elements_by_tag_name('tr')])

    # Convert data to DataFrame with column
    df = pd.DataFrame(data, columns = column_name)
    
    # Add search stock data and date_to_search
    df['stock_code'] = stock_code
    df['report_date'] = date_to_search
    
    return df
    

In [63]:
pd.DataFrame(data, columns=column_name)

,參與者編號,中央結算系統參與者名稱\n(*即願意披露的投資者戶口持有人),地址,持股量,佔已發行股份/權證/單位百分比
0,C00019,香港上海匯豐銀行有限公司,HSBC WEALTH BUSINESS SERVICES 8/F TOWER 2 & 3 ...,"1,158,593,273",30.04%
1,C00100,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",18/F TOWER 2 THE QUAYSIDE 77 HOI BUN ROAD KWUN...,"360,287,111",9.34%
2,C00010,花旗銀行,9/F CITI TOWER ONE BAY EAST 83 HOI BUN ROAD KW...,"246,218,925",6.38%
3,C00039,渣打銀行(香港)有限公司,SECURITIES SERVICES 15/F STANDARD CHARTERED TO...,"138,825,795",3.60%
4,C00033,中國銀行(香港)有限公司,11/F BANK OF CHINA CENTRE OLYMPIAN CITY 11 HOI...,"72,355,334",1.87%
...,...,...,...,...,...
431,B02022,潮商證券有限公司,ROOM 2206-2210 CHINA RESOURCES BUILDING 26 HAR...,500,0.00%
432,B02060,力高證券有限公司,ROOM 301 3/F CHINA BUILDING 29 QUEEN'S ROAD CE...,500,0.00%
433,B02075,創陞證券有限公司,UNIT A-C 20/F NEICH TOWER 128 GLOUCESTER ROAD ...,500,0.00%
434,B01824,極訊亞太有限公司,31/F TWO INTERNATIONAL FINANCE CENTRE 8 FINANC...,400,0.00%


In [60]:
search_body.find_elements_by_tag_name('tr')[0].find_elements_by_class_name('mobile-list-body')

[<selenium.webdriver.remote.webelement.WebElement (session="4cfeb1badc9383ad7f1ab61ed0348cc6", element="adb0bc0a-558d-4485-b95d-f863fae113a7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4cfeb1badc9383ad7f1ab61ed0348cc6", element="ac053a73-55cb-44d8-9290-10b6a1ef086c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4cfeb1badc9383ad7f1ab61ed0348cc6", element="7257ccca-df36-48f7-910f-eb7380492b3d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4cfeb1badc9383ad7f1ab61ed0348cc6", element="efd356f3-03f2-438d-951f-2112df8f3a89")>,
 <selenium.webdriver.remote.webelement.WebElement (session="4cfeb1badc9383ad7f1ab61ed0348cc6", element="f39a192e-9fe5-4733-b6ec-591dcee2aca0")>]

In [19]:
import pandas as pd

df = pd.read_csv('reference/ListOfSecurities.csv')
df = df[df['Stock Code'].isnull() == False]

In [23]:
# Convert df to json
result_df = df['Stock Code'].apply(lambda x: ('00000' + str(int(x)))[-5:])

In [26]:
result_df.to_json('reference/ListOfStock.json')

ValueError: 'index=False' is only valid when 'orient' is 'split' or 'table'

In [27]:
import json

In [37]:
path = 'reference/ListOfStock.json'
get_stock_codes(path)

['00001',
 '00002',
 '00003',
 '00004',
 '00005',
 '00006',
 '00007',
 '00008',
 '00009',
 '00010',
 '00011',
 '00012',
 '00014',
 '00016',
 '00017',
 '00018',
 '00019',
 '00021',
 '00022',
 '00023',
 '00024',
 '00025',
 '00026',
 '00027',
 '00028',
 '00029',
 '00030',
 '00031',
 '00032',
 '00033',
 '00034',
 '00035',
 '00036',
 '00037',
 '00038',
 '00039',
 '00040',
 '00041',
 '00042',
 '00043',
 '00045',
 '00046',
 '00047',
 '00048',
 '00050',
 '00051',
 '00052',
 '00053',
 '00055',
 '00056',
 '00057',
 '00058',
 '00059',
 '00060',
 '00061',
 '00062',
 '00063',
 '00064',
 '00065',
 '00066',
 '00067',
 '00068',
 '00069',
 '00070',
 '00071',
 '00072',
 '00073',
 '00075',
 '00076',
 '00077',
 '00078',
 '00079',
 '00080',
 '00081',
 '00082',
 '00083',
 '00084',
 '00085',
 '00086',
 '00087',
 '00088',
 '00089',
 '00090',
 '00091',
 '00092',
 '00093',
 '00094',
 '00095',
 '00096',
 '00097',
 '00098',
 '00099',
 '00100',
 '00101',
 '00102',
 '00103',
 '00104',
 '00105',
 '00106',
 '00107',
